# NN Model

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse 
# local imports
from model_tools import PredictorData
from nn_models import TF_NN_Predictor
from utils import Utils

2024-06-09 16:46:51.095830: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 16:46:51.103093: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 16:46:51.171203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 16:46:52.937119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
config = Utils.read_config_for_env(config_path='../config/config.yml')
pred_data = PredictorData(
    config,
    refresh_monthly=False,
    refresh_ts_features=False,
    clean_strategy='olrem_for_all',
    # split_strategy='random',
    # split_strategy='last_months_val',
    split_strategy='months',
    num_lag_mon=3,
    val_ratio=0.2)

Loading data.. Done.
Fixing data schemas.. Done.
Loading /home/onur/WORK/DS/repos/shop_sales_prediction/data/all_ts.parquet


## NN model with 4 Dense layers

In [3]:
tf_nn_predictor_4D = TF_NN_Predictor(
    pred_data=pred_data,
    output_dim=1,
    model_name='4Dense',
    optimizer="adam",
    metrics=['mse'])

In [4]:
tf_nn_predictor_4D.pred_data = tf_nn_predictor_4D.split_transform(
    tf_nn_predictor_4D.pred_data,
    tf_nn_predictor_4D.transformer)

Splitting train-val
Fit-transforming X_train
Transforming X_val


In [6]:
tf_nn_predictor_4D.create_model()

Creating model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 24)             │         1,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,209 (20.35 KB)

 Trainable params: 5,209 (20.35 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
#Train the model
tf_nn_predictor_4D.model.fit(
    tf_nn_predictor_4D.pred_data.X_train,
    tf_nn_predictor_4D.pred_data.y_train,
    epochs=10,
    batch_size=4096,
    validation_data=(tf_nn_predictor_4D.pred_data.X_val, tf_nn_predictor_4D.pred_data.y_val),
    callbacks=tf_nn_predictor_4D.callbacks)

2024-06-09 16:48:15.322828: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2394360000 exceeds 10% of free system memory.


Epoch 1/10
8116/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3369 - mse: 0.3369
Epoch 1: val_loss improved from inf to 0.39882, saving model to 4Dense_weights-001--0.39882.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 94s 11ms/step - loss: 0.3369 - mse: 0.3369 - val_loss: 0.3988 - val_mse: 0.3988
Epoch 2/10
8116/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2954 - mse: 0.2954
Epoch 2: val_loss improved from 0.39882 to 0.39840, saving model to 4Dense_weights-002--0.39840.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 91s 11ms/step - loss: 0.2954 - mse: 0.2954 - val_loss: 0.3984 - val_mse: 0.3984
Epoch 3/10
8114/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3011 - mse: 0.3011
Epoch 3: val_loss improved from 0.39840 to 0.39319, saving model to 4Dense_weights-003--0.39319.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 91s 11ms/step - loss: 0.3011 - mse: 0.3011 - val_loss: 0.3932 - val_mse: 0.3932
Epoch 4/10
8115/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2933 - mse: 0.2933
Epoch 4: val_loss improved fr

In [10]:
# Make predictions on the training and validation data. 
# pred_train = tf_nn_predictor_4D.model.predict_on_batch(pred_data.X_train)
pred_val = tf_nn_predictor_4D.model.predict_on_batch(pred_data.X_val)
# pred_train = tf_nn_predictor_4D.model.predict(pred_data.X_train)
# pred_val = tf_nn_predictor_4D.model.predict(pred_data.X_val)

# Calculate and print the Root Mean Squared Error (RMSE) for training and validation predictions. 
# print("Training RMSE: ", np.sqrt(mse(pred_data.y_train, pred_train)))
print("Validation RMSE: ", np.sqrt(mse(pred_data.y_val, pred_val)))

Validation RMSE:  0.6260533
